In [90]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from patsy import dmatrices
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [91]:
import os
os.getcwd()

'C:\\Users\\dave_\\Desktop\\DHDA\\SEMESTER2\\Data Analytics II\\Week 6 - logistic regression'

In [92]:
xtrcovbs = pd.read_sas('xtrfxcovbs.sas7bdat', format='sas7bdat')

In [93]:
xtrcovbs.columns = xtrcovbs.columns.str.lower()
xtrcovbs.columns

Index(['replicate', 't_xtrdate', 'r_xtrdate', 'nonewosteofx_enddate',
       'nonewhipfx_enddate', 'qid', 't_fl_fea', 't_ctpo_ec', 't_ttar',
       't_ctar', 't_tbar', 't_ttbmd', 't_ctbmd', 't_ctth', 't_tbbmd', 't_tbn',
       't_tbth', 't_tbsp', 't_moart', 'r_fl_fea', 'r_ctpo_ec', 'r_ttar',
       'r_ctar', 'r_tbar', 'r_ttbmd', 'r_ctbmd', 'r_ctth', 'r_tbbmd', 'r_tbn',
       'r_tbth', 'r_tbsp', 'r_moart', 't_bvtv', 'r_bvtv', 'fnewopfxd',
       'fnewhipfxd', 'newosteofx', 'firstnewosteofx_site',
       'firstnewosteofx_year', 'firstnewosteofx_month', 'firstnewosteofx_day',
       'newosteofx_trauma', 'newhipfx', 'firstnewhipfx_year',
       'firstnewhipfx_month', 'firstnewhipfx_day', 'newhipfx_trauma',
       'nonewosteofx_enddispo', 'nonewhipfx_enddispo', 'age', 'height',
       'weight', 'centre', 'parenthipfx', 'prevfx_any', 'cig_cur',
       'nbalc_week', 'rheu_ever', 'cortico_oral_reg', 'secop', 'nbfall_pastyr',
       'everlost10lbs', 'osteomed', 'i', 'prev_backmt', 'prev_hipmt'

In [94]:
#drop all rows with missing values for the variables you are looking at
xtrfx = xtrcovbs[['t_tbsp','age',"height", "weight","newosteofx"]].dropna()

np.isnan(xtrfx).value_counts()

t_tbsp  age    height  weight  newosteofx
False   False  False   False   False         1192
dtype: int64

In [95]:
#sklearn
x = xtrfx[["t_tbsp"]] # 2D array
y = xtrfx["newosteofx"]

logmod = LogisticRegression(fit_intercept = True, penalty='none', solver='newton-cg')
fxmod = logmod.fit(x, y)

In [96]:
# Then to get results: 
r_sq = fxmod.score(x, y)
intercept, coefficients = fxmod.intercept_, fxmod.coef_

# NOTE: that by default, just like in SAS & R, the intercept is modeled here 

In [97]:
r_sq, intercept, coefficients

(0.8976510067114094, array([-3.0578872]), array([[1.61227097]]))

In [98]:
# odds ratio for age:
np.exp(coefficients[0][0])

# note: unfortunately there is no easy way to generate confidence intervals in sklearn

5.014185358679075

#statsmodel

In [99]:
#statsmodel
#outcome - newosteofx
#exposure - t_tbsp
#covariates - age, bmi
xtrfx = xtrcovbs[["age", "height", "weight", "newosteofx", 't_tbsp']].dropna()
xtrfx['bmi'] = xtrfx.weight / (xtrfx.height / 100) **2
x = xtrfx[["t_tbsp", "age", "bmi"]] # 2D array
y = xtrfx["newosteofx"]

In [100]:
# need to manually add a constant 1.0 to get intercept		 
# NOTE: unlike SAS, R, and scikitlearn, this step needs to be explicit
x1 = sm.add_constant(x)

In [101]:
fxmod2 = sm.Logit(y, x1)
model2fit = fxmod2.fit()

print(model2fit.summary())

Optimization terminated successfully.
         Current function value: 0.321166
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             newosteofx   No. Observations:                 1192
Model:                          Logit   Df Residuals:                     1188
Method:                           MLE   Df Model:                            3
Date:                Wed, 23 Feb 2022   Pseudo R-squ.:                 0.01650
Time:                        15:11:06   Log-Likelihood:                -382.83
converged:                       True   LL-Null:                       -389.25
Covariance Type:            nonrobust   LLR p-value:                  0.004977
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2175      1.143     -3.690      0.000      -6.458      -1.977
t_tbsp         1.5448      0.

In [102]:
model2fit.params

const    -4.217527
t_tbsp    1.544817
age       0.016667
bmi      -0.001058
dtype: float64

In [103]:
round(np.exp(model2fit.params),2)

const     0.01
t_tbsp    4.69
age       1.02
bmi       1.00
dtype: float64

In [104]:
orparm = model2fit.params
conf = model2fit.conf_int()

# add column for odds ratio (OR) determined above
conf['Odds Ratio'] = orparm
conf.columns = ['5% CI', '95% CI', 'Odds Ratio'] # change column names 

ortable = round(np.exp(conf),2)
ortable

,5% CI,95% CI,Odds Ratio
const,0.00,0.14,0.01
t_tbsp,1.89,11.65,4.69
age,0.99,1.04,1.02
bmi,0.96,1.04,1.00


redo expressed as 1 SD change

In [105]:
x1b.head()

,const,t_tbsp,age,bmi
0,1.0,3.568733,75.0,25.269310
1,1.0,3.568733,75.0,25.269310
2,1.0,2.694170,69.0,33.628662
3,1.0,3.415516,69.0,26.657922
4,1.0,2.980992,78.0,27.452494


In [106]:
np.std(x1b['t_tbsp'])

1.000000000000001

In [107]:
x1b = x1.copy()
x1b['t_tbsp'] = x1b['t_tbsp']/np.std(x1b['t_tbsp'])

In [108]:
fxmod2b = sm.Logit(y, x1b)
model2bfit = fxmod2b.fit()

print(model2bfit.summary())

Optimization terminated successfully.
         Current function value: 0.321166
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             newosteofx   No. Observations:                 1192
Model:                          Logit   Df Residuals:                     1188
Method:                           MLE   Df Model:                            3
Date:                Wed, 23 Feb 2022   Pseudo R-squ.:                 0.01650
Time:                        15:11:06   Log-Likelihood:                -382.83
converged:                       True   LL-Null:                       -389.25
Covariance Type:            nonrobust   LLR p-value:                  0.004977
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2175      1.143     -3.690      0.000      -6.458      -1.977
t_tbsp         0.2521      0.

In [109]:
ortable = round(np.exp(model2bfit.params),2)
ortable

const     0.01
t_tbsp    1.29
age       1.02
bmi       1.00
dtype: float64

In [110]:
orparmb = model2bfit.params
confb = model2bfit.conf_int()

# add column for odds ratio (OR) determined above
confb['Odds Ratio'] = orparmb
confb.columns = ['5%', '95%', 'Odds Ratio'] # change column names 

ortableb = round(np.exp(confb),2)
ortableb

,5%,95%,Odds Ratio
const,0.00,0.14,0.01
t_tbsp,1.11,1.49,1.29
age,0.99,1.04,1.02
bmi,0.96,1.04,1.00


#set categorical exposure variable by creating ref variable 

In [111]:
import patsy # when using a categorical exposure, need to use formula instead of a list of variables

In [112]:
xtrcovbs2 = xtrcovbs.copy()

In [113]:
# making sure no missing values
xtrfx = xtrcovbs2[["t_ctth",'age', "height", "weight", "newosteofx"]].dropna()
#code bmi 
xtrfx['bmi'] = xtrfx.weight / (xtrfx.height / 100) **2

#code t_ctth_cat 
xtrfx.loc[xtrfx['t_ctth'] < 0.77, 't_ctth_cat'] = 1   
xtrfx.loc[xtrfx['t_ctth'] < 0.96, 't_ctth_cat'] = 2 
xtrfx.loc[xtrfx['t_ctth'] < 1.12, 't_ctth_cat'] = 3
xtrfx.loc[xtrfx['t_ctth'] > 1.12, 't_ctth_cat'] = 4

y1c, x1c = patsy.dmatrices("newosteofx ~ C(t_ctth_cat) + age + bmi",
                        data=xtrfx,
                        return_type="dataframe")

In [114]:
x1c

,Intercept,C(t_ctth_cat)[T.4.0],age,bmi
0,1.0,0.0,75.0,25.269310
1,1.0,0.0,75.0,25.269310
2,1.0,0.0,69.0,33.628662
3,1.0,1.0,69.0,26.657922
4,1.0,1.0,78.0,27.452494
...,...,...,...,...
1226,1.0,0.0,70.0,24.236686
1227,1.0,0.0,70.0,24.236686
1228,1.0,0.0,70.0,24.236686
1229,1.0,0.0,70.0,24.236686


In [115]:
fxmod2c = sm.Logit(y1c, x1c)
model2cfit = fxmod2c.fit()

# If no referent defined, always uses first category
print(model2cfit.summary())

Optimization terminated successfully.
         Current function value: 0.323294
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             newosteofx   No. Observations:                 1189
Model:                          Logit   Df Residuals:                     1185
Method:                           MLE   Df Model:                            3
Date:                Wed, 23 Feb 2022   Pseudo R-squ.:                 0.01167
Time:                        15:11:06   Log-Likelihood:                -384.40
converged:                       True   LL-Null:                       -388.93
Covariance Type:            nonrobust   LLR p-value:                   0.02829
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -2.8386      1.114     -2.547      0.011      -5.023      -0.655

In [116]:
orparmc = model2cfit.params
confc = model2cfit.conf_int()

# add column for odds ratio (OR) determined above
confc['Odds Ratio'] = orparmc
confc.columns = ['5%', '95%', 'Odds Ratio'] # change column names 

ortablec = round(np.exp(confc),2)
ortablec

,5%,95%,Odds Ratio
Intercept,0.01,0.52,0.06
C(t_ctth_cat)[T.4.0],0.31,0.90,0.53
age,0.99,1.04,1.01
bmi,0.96,1.03,1.00


REPEAT WITH REFERENT

In [120]:
y1d, x1d = patsy.dmatrices("newosteofx ~ C(t_ctth_cat, Treatment(reference=3)) + height + weight",
                        data=xtrfx,
                        return_type="dataframe")

In [1]:
data.rheu_ever

NameError: name 'data' is not defined

In [121]:
fxmod2d = sm.Logit(y1d, x1d)
model2dfit = fxmod2d.fit()

print(model2dfit.summary())

Optimization terminated successfully.
         Current function value: 0.322964
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             newosteofx   No. Observations:                 1189
Model:                          Logit   Df Residuals:                     1185
Method:                           MLE   Df Model:                            3
Date:                Wed, 23 Feb 2022   Pseudo R-squ.:                 0.01268
Time:                        15:13:02   Log-Likelihood:                -384.00
converged:                       True   LL-Null:                       -388.93
Covariance Type:            nonrobust   LLR p-value:                   0.01979
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                               

In [122]:
# Now generating odds ratios: 

orparmd = model2dfit.params
confd = model2dfit.conf_int()

# add column for odds ratio (OR) determined above
confd['Odds Ratio'] = orparmd
confd.columns = ['5%', '95%', 'Odds Ratio'] # change column names 

ortabled = round(np.exp(confd),2)
ortabled

,5%,95%,Odds Ratio
Intercept,0.00,0.73,0.01
"C(t_ctth_cat, Treatment(reference=3))[T.4.0]",0.31,0.88,0.53
height,0.99,1.05,1.02
weight,0.98,1.01,1.00
